# Google Provider Guide 🌟

This notebook demonstrates how to use Google's Gemini models through different hosting options:
- **AI Studio**: Google's direct API service for developers
- **Vertex AI**: Enterprise-grade platform through Google Cloud

## Overview

Google offers Gemini models through multiple platforms, each designed for different use cases:

### AI Studio (Direct Google Hosting)
- **Easy setup** with simple API key authentication
- **Latest models** available immediately upon release
- **Developer-friendly** with minimal configuration
- **Free tier** available for experimentation
- **Global access** without regional restrictions

### Vertex AI (Google Cloud Hosting)
- **Enterprise features** including VPC integration
- **Regional deployment** for compliance and latency
- **Advanced monitoring** and logging capabilities
- **Custom model fine-tuning** options
- **IAM integration** for access control
- **Batch processing** and high-throughput scenarios

## Setup

First, let's load environment variables for authentication:

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## AI Studio (Direct Google Hosting)

### Authentication
AI Studio uses the `GEMINI_API_KEY` environment variable for authentication. You can get your API key from [Google AI Studio](https://aistudio.google.com/).

### Available Models
- `gemini-2.0-flash-001` - Latest and fastest Gemini model
- `gemini-1.5-pro-001` - Most capable model for complex tasks
- `gemini-1.5-flash-001` - Balanced performance and speed
- `gemini-1.0-pro` - Previous generation model

### Key Features
- **Multimodal capabilities** (text, images, audio, video)
- **Function calling** for tool integration
- **Large context windows** (up to 2M tokens)
- **Code generation** and execution
- **Reasoning capabilities** with chain-of-thought

### Resources
- [Python SDK Documentation](https://github.com/googleapis/python-genai)
- [AI Studio](https://aistudio.google.com/)
- [Model Documentation](https://ai.google.dev/models/gemini)

### Basic Message Example

In [3]:
from google import genai

# Initialize AI Studio client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Basic content generation
response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='Explain quantum computing in simple terms, using an analogy.'
)

print("=== AI Studio Response ===")
print(f"Model: {response.model_version}")

if response.candidates and len(response.candidates) > 0:
    candidate = response.candidates[0]
    print(f"Content: {candidate.content.parts[0].text}")
    print(f"Finish reason: {candidate.finish_reason}")
    
    # Safety ratings (if available)
    if candidate.safety_ratings:
        print(f"Safety ratings: {len(candidate.safety_ratings)} categories checked")

print(f"Usage: {response.usage_metadata.prompt_token_count} + {response.usage_metadata.candidates_token_count} = {response.usage_metadata.total_token_count} tokens")

# Full response structure
print("\n=== Full Response Structure ===")
response.model_dump()

=== AI Studio Response ===
Model: gemini-2.0-flash-001
Content: Okay, imagine you're trying to find the best way to get through a maze.

**Classical Computing is like:**

You're a single person, and you try one path at a time. You go down one route, hit a dead end, backtrack, and try another route. You have to explore each path individually until you find the right one.

**Quantum Computing is like:**

Instead of you being one person, you're a **super-powered ghost** that can exist in multiple places *at the same time*. You can explore **all possible paths through the maze simultaneously**.  You're not just going down one route; you're going down *every* route, *at the same time*, as a probability.

*   **Classical bit:** Like a light switch - either ON (1) or OFF (0).
*   **Qubit (quantum bit):** Like a dimmer switch. It can be ON (1), OFF (0), or somewhere *in between* (represented as a combination of both, called a *superposition*).

Because of this "in-between" state, qubits can ho

{'candidates': [{'content': {'parts': [{'video_metadata': None,
      'thought': None,
      'inline_data': None,
      'code_execution_result': None,
      'executable_code': None,
      'file_data': None,
      'function_call': None,
      'function_response': None,
      'text': 'Okay, imagine you\'re trying to find the best way to get through a maze.\n\n**Classical Computing is like:**\n\nYou\'re a single person, and you try one path at a time. You go down one route, hit a dead end, backtrack, and try another route. You have to explore each path individually until you find the right one.\n\n**Quantum Computing is like:**\n\nInstead of you being one person, you\'re a **super-powered ghost** that can exist in multiple places *at the same time*. You can explore **all possible paths through the maze simultaneously**.  You\'re not just going down one route; you\'re going down *every* route, *at the same time*, as a probability.\n\n*   **Classical bit:** Like a light switch - either ON (

### Advanced Configuration with Messages

For more complex interactions, you can use the message format with system instructions:

In [4]:
from google.genai import types

# Create a conversation with system instructions
system_instruction = "You are a helpful AI assistant specializing in creative writing. Always provide vivid, engaging responses."

messages = [
    types.Content(
        role="user",
        parts=[types.Part(text="Write a compelling opening paragraph for a mystery novel set in a futuristic city.")]
    )
]

# Configuration for more control
config = types.GenerateContentConfig(
    system_instruction=system_instruction,
    temperature=0.8,  # More creative responses
    top_p=0.95,
    max_output_tokens=200,
    candidate_count=1
)

advanced_response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=messages,
    config=config
)

print("=== Advanced Configuration Response ===")
print(f"Model: {advanced_response.model_version}")
if advanced_response.candidates:
    print(f"Content: {advanced_response.candidates[0].content.parts[0].text}")
    print(f"Average log probability: {advanced_response.candidates[0].avg_logprobs:.4f}")

=== Advanced Configuration Response ===
Model: gemini-2.0-flash-001
Content: The neon rain of Neo-Kyoto slicked the chrome streets, reflecting a kaleidoscope of fractured light onto Detective Kaito Ishikawa's trench coat. He stood on the precipice of the Sky-Gardens, the artificial flora shimmering unnaturally under the electric downpour, a stark contrast to the lifeless corpse sprawled at his feet. It wasn't the synthetic cherry blossoms raining down on the scene, nor the panoramic view of the sprawling cityscape that caught his attention, but the single, archaic katana plunged through the victim's chest, a rusty relic in a world of laser pistols and bio-engineered weaponry. This was more than just a murder; it was a message, scrawled in blood and steel, a ghost from the past haunting the hyper-modern present.

Average log probability: -0.3911


## Function Calling (Tool Use)

Google's Gemini models support function calling for interacting with external tools and APIs.

### Key Features:
- **Function declarations** using JSON schemas
- **Parallel function calls** for efficiency
- **Automatic function calling** mode
- **Complex parameter schemas** with nested objects

### Single Function Call Example

In [5]:
# Define a weather function
weather_function = {
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Tokyo, Japan"
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Temperature units"
            }
        },
        "required": ["location"]
    }
}

# Create function tool
tools = types.Tool(function_declarations=[weather_function])
config = types.GenerateContentConfig(tools=[tools])

# Initial request with function
messages = [
    types.Content(
        role="user",
        parts=[types.Part(text="What is the weather like in Tokyo today?")]
    )
]

function_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=messages,
    config=config
)

print("=== Function Call Response ===")
print(f"Model: {function_response.model_version}")

if function_response.candidates and len(function_response.candidates) > 0:
    candidate = function_response.candidates[0]
    
    for i, part in enumerate(candidate.content.parts):
        print(f"\nPart {i+1}: {part.__class__.__name__}")
        
        if part.text:
            print(f"  Text: {part.text}")
        elif part.function_call:
            print(f"  Function: {part.function_call.name}")
            print(f"  Arguments: {dict(part.function_call.args)}")

# Add assistant's response to conversation
messages.append(candidate.content)

print(f"\nMessages so far: {len(messages)}")

=== Function Call Response ===
Model: gemini-2.0-flash

Part 1: Part
  Function: get_weather
  Arguments: {'location': 'Tokyo, Japan'}

Messages so far: 2


### Function Execution and Response

In [6]:
# Simulate function execution
def execute_weather_function(location, units="celsius"):
    """Simulate getting weather data"""
    if "tokyo" in location.lower():
        if units == "celsius":
            return "22°C, partly cloudy with light breeze"
        else:
            return "72°F, partly cloudy with light breeze"
    else:
        return f"Weather data not available for {location}"

# Find function call in the response
function_call = None
if function_response.candidates:
    for part in function_response.candidates[0].content.parts:
        if part.function_call:
            function_call = part.function_call
            break

if function_call:
    # Execute the function
    location = function_call.args.get("location")
    units = function_call.args.get("units", "celsius")
    weather_result = execute_weather_function(location, units)
    
    print(f"=== Function Execution ===")
    print(f"Function: {function_call.name}")
    print(f"Location: {location}")
    print(f"Units: {units}")
    print(f"Result: {weather_result}")
    
    # Create function response
    function_response_part = types.Part.from_function_response(
        name=function_call.name,
        response={"result": weather_result}
    )
    
    messages.append(
        types.Content(role="user", parts=[function_response_part])
    )
    
    # Get final response with function results
    final_response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=messages,
        config=config
    )
    
    print(f"\n=== Final Response ===")
    if final_response.candidates:
        print(f"Gemini's response: {final_response.candidates[0].content.parts[0].text}")
else:
    print("No function calls found in response")

=== Function Execution ===
Function: get_weather
Location: Tokyo, Japan
Units: celsius
Result: 22°C, partly cloudy with light breeze

=== Final Response ===
Gemini's response: It is 22°C in Tokyo, partly cloudy with a light breeze.



### Multiple Functions Example

In [7]:
# Define multiple functions
calculator_function = {
    "name": "calculator",
    "description": "Perform basic mathematical operations",
    "parameters": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The mathematical operation"
            },
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
        "required": ["operation", "a", "b"]
    }
}

time_function = {
    "name": "get_current_time",
    "description": "Get the current time in a specified timezone",
    "parameters": {
        "type": "object",
        "properties": {
            "timezone": {
                "type": "string",
                "description": "Timezone (e.g., UTC, America/New_York, Asia/Tokyo)",
                "default": "UTC"
            }
        },
        "required": []
    }
}

# Create tools with multiple functions
multi_tools = types.Tool(function_declarations=[
    weather_function,
    calculator_function,
    time_function
])
multi_config = types.GenerateContentConfig(tools=[multi_tools])

# Request that might use multiple functions
multi_function_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text="Calculate 35 * 12, and also tell me what time it is in Tokyo")]
        )
    ],
    config=multi_config
)

print("=== Multiple Functions Response ===")
print(f"Model: {multi_function_response.model_version}")

if multi_function_response.candidates:
    candidate = multi_function_response.candidates[0]
    
    print(f"\nContent parts: {len(candidate.content.parts)}")
    function_calls = []
    
    for i, part in enumerate(candidate.content.parts):
        print(f"\nPart {i+1}: {part.__class__.__name__}")
        
        if part.text:
            print(f"  Text: {part.text}")
        elif part.function_call:
            print(f"  Function: {part.function_call.name}")
            print(f"  Arguments: {dict(part.function_call.args)}")
            function_calls.append(part.function_call)
    
    print(f"\nTotal function calls: {len(function_calls)}")
    print(f"Usage: {multi_function_response.usage_metadata.prompt_token_count} + {multi_function_response.usage_metadata.candidates_token_count} = {multi_function_response.usage_metadata.total_token_count} tokens")

=== Multiple Functions Response ===
Model: gemini-2.0-flash

Content parts: 2

Part 1: Part
  Function: calculator
  Arguments: {'operation': 'multiply', 'b': 12, 'a': 35}

Part 2: Part
  Function: get_current_time
  Arguments: {'timezone': 'Asia/Tokyo'}

Total function calls: 2
Usage: 113 + 16 = 129 tokens


## Images and PDF-files

### Images
First let's try sending an image from local storage

In [8]:
import requests

image1_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"

# Download image from URL instead of trying to open it as a local file
response = requests.get(image1_url)
image_bytes = response.content

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=[
        types.Part.from_bytes(
            data=image_bytes,
            mime_type='image/jpeg',
        ),
        "What's this image about? "
    ]
)

print(response.text)

The image shows a close-up of an ant standing on a rough surface. The ant is dark in color, with visible segments on its body and long, thin legs. Its antennae are also noticeable. The background is blurry, with warm tones of brown and red. The depth of field is shallow, focusing attention on the ant.



You can also just pass a URL if it's a public image

In [9]:
image_path = "https://goo.gle/instrument-img"
image_bytes = requests.get(image_path).content
image = types.Part.from_bytes(
  data=image_bytes, mime_type="image/jpeg"
)

response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=["What is this image?", image],
)

print(response.text)

The image shows a pipe organ console. It has multiple keyboards (manuals), rows of stops, foot pedals, and other controls.



### PDF-Files

In [12]:
prompt = "Summarize this document"
response = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[
      types.Part.from_bytes(
        data=open("./assets/gameboy_color.pdf", "rb").read(),
        mime_type='application/pdf',
      ),
      prompt])
print(response.text)

This document is the instruction manual for the Nintendo Game Boy Color. It outlines the components of the device, how to install batteries, use Game Paks, change the screen color, and connect with other Game Boy systems using the Game Link cable or the infrared communication port. It also provides troubleshooting tips, warranty information, and a parts list with an order form. The manual emphasizes safety and compatibility and directs users to contact Nintendo's Consumer Assistance Hotline or authorized repair centers for additional help.



In [11]:
myfile = client.files.upload(file="./assets/gameboy_color.pdf")

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=["Describe this PDF in 2 sentences", myfile]
)

print(response.text)

This is the instruction booklet for the Game Boy Color, outlining the features of the device, how to use it, and troubleshooting steps. It also includes warranty information, and a parts list and order form for replacement parts or accessories.



## Vertex AI Hosting

### Setup and Authentication
Vertex AI requires Google Cloud project configuration and uses service account authentication or Application Default Credentials (ADC).

### Key Differences:
- **Project ID**: Must specify your Google Cloud project
- **Location**: Choose deployment region for compliance/latency
- **Enterprise Features**: VPC integration, audit logging, and advanced monitoring
- **IAM Integration**: Fine-grained access control
- **Custom Models**: Support for fine-tuned and custom models

### Available Regions:
- `us-central1` - United States (primary)
- `us-east4` - United States (secondary)
- `europe-west1` - Europe (primary)
- `asia-northeast1` - Asia Pacific (Tokyo)
- See [Vertex AI Locations](https://cloud.google.com/vertex-ai/docs/general/locations) for complete list

### Resources:
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai/docs)
- [Gemini on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini)
- [Google Cloud Console](https://console.cloud.google.com/vertex-ai)

### Basic Vertex AI Example

In [8]:
# Initialize Vertex AI client
project_id = os.environ.get("GCP_PROJECT_ID")
location = os.environ.get("GCP_LOCATION", "us-central1")

print(f"Vertex AI Configuration:")
print(f"  Project: {project_id}")
print(f"  Location: {location}")

vertex_client = genai.Client(
    vertexai=True,
    project=project_id,
    location=location
)

# Basic content generation on Vertex AI
vertex_response = vertex_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='Explain the benefits of using Vertex AI over AI Studio for enterprise applications.'
)

print("\n=== Vertex AI Response ===")
print(f"Model: {vertex_response.model_version}")

if vertex_response.candidates and len(vertex_response.candidates) > 0:
    candidate = vertex_response.candidates[0]
    print(f"Content: {candidate.content.parts[0].text}")
    print(f"Finish reason: {candidate.finish_reason}")

print(f"Usage: {vertex_response.usage_metadata.prompt_token_count} + {vertex_response.usage_metadata.candidates_token_count} = {vertex_response.usage_metadata.total_token_count} tokens")

# Additional Vertex AI metadata
if hasattr(vertex_response, 'create_time'):
    print(f"Create time: {vertex_response.create_time}")
if hasattr(vertex_response, 'response_id'):
    print(f"Response ID: {vertex_response.response_id}")

print("\n=== Full Response Structure ===")
vertex_response

Vertex AI Configuration:
  Project: vectrix-401014
  Location: europe-west1

=== Vertex AI Response ===
Model: gemini-2.0-flash-001
Content: While both Vertex AI and AI Studio (formerly known as Colab Enterprise) offer tools for building and deploying machine learning models, **Vertex AI is generally the preferred choice for enterprise applications due to its comprehensive features, scalability, and governance capabilities.**  Here's a breakdown of the benefits:

**1. End-to-End ML Lifecycle Support:**

*   **Vertex AI:** Designed to manage the entire ML lifecycle, from data preparation and model training to deployment, monitoring, and continuous improvement. It provides a unified platform for all stages, promoting collaboration and efficiency.
*   **AI Studio:** Primarily focuses on interactive development and experimentation. While it offers some deployment capabilities, it's not as comprehensive or geared towards managing production-level ML pipelines.

**2. Scalability and Reliabil

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text="While both Vertex AI and AI Studio (formerly known as Colab Enterprise) offer tools for building and deploying machine learning models, **Vertex AI is generally the preferred choice for enterprise applications due to its comprehensive features, scalability, and governance capabilities.**  Here's a breakdown of the benefits:\n\n**1. End-to-End ML Lifecycle Support:**\n\n*   **Vertex AI:** Designed to manage the entire ML lifecycle, from data preparation and model training to deployment, monitoring, and continuous improvement. It provides a unified platform for all stages, promoting collaboration and efficiency.\n*   **AI Studio:** Primarily focuses on interactive development and experimentation. While it offers some deployment capabilities, it's no

### Vertex AI Function Calling

Function calling works identically on Vertex AI as with AI Studio:

In [9]:
# Function calling works the same on Vertex AI
vertex_tools = types.Tool(function_declarations=[calculator_function])
vertex_config = types.GenerateContentConfig(tools=[vertex_tools])

vertex_function_response = vertex_client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text="Calculate 88 divided by 11")]
        )
    ],
    config=vertex_config
)

print("=== Vertex AI Function Call ===")
print(f"Model: {vertex_function_response.model_version}")

if vertex_function_response.candidates:
    candidate = vertex_function_response.candidates[0]
    
    for part in candidate.content.parts:
        if part.text:
            print(f"Text: {part.text}")
        elif part.function_call:
            print(f"Function: {part.function_call.name}")
            args = dict(part.function_call.args)
            print(f"Operation: {args['a']} {args['operation']} {args['b']}")
            print(f"Arguments: {args}")

print(f"\n✅ Function calling works identically on Vertex AI!")

=== Vertex AI Function Call ===
Model: gemini-2.0-flash-001
Function: calculator
Operation: 88 divide 11
Arguments: {'operation': 'divide', 'a': 88, 'b': 11}

✅ Function calling works identically on Vertex AI!


## Comparison: AI Studio vs Vertex AI

Let's compare the same request across both hosting options:

In [11]:
test_content = "Explain artificial intelligence in exactly 2 sentences."

print("=== Comparison: AI Studio vs Vertex AI ===")

# AI Studio
studio_response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=test_content
)

print("\n🟡 AI Studio:")
print(f"Model: {studio_response.model_version}")
if studio_response.candidates:
    print(f"Response: {studio_response.candidates[0].content.parts[0].text}")
print(f"Tokens: {studio_response.usage_metadata.prompt_token_count} + {studio_response.usage_metadata.candidates_token_count}")

# Vertex AI
vertex_response = vertex_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=test_content
)

print("\n🔵 Vertex AI:")
print(f"Model: {vertex_response.model_version}")
if vertex_response.candidates:
    print(f"Response: {vertex_response.candidates[0].content.parts[0].text}")
print(f"Tokens: {vertex_response.usage_metadata.prompt_token_count} + {vertex_response.usage_metadata.candidates_token_count}")

print("\n💡 Both use identical APIs and response formats!")
print("💡 Key differences: authentication, enterprise features, and regional deployment")

=== Comparison: AI Studio vs Vertex AI ===

🟡 AI Studio:
Model: gemini-2.0-flash-001
Response: Artificial intelligence is the development of computer systems capable of performing tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. These systems often achieve this through algorithms and models that allow them to analyze data, identify patterns, and adapt to new information.

Tokens: 9 + 57

🔵 Vertex AI:
Model: gemini-2.0-flash-001
Response: Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, and problem-solving, enabling machines to perform tasks that typically require human intelligence.

Tokens: 9 + 40

💡 Both use identical APIs and response formats!
💡 Key differences: authentication, enterprise features, and regional deployment
